In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('AirPort').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/08 09:23:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
port_df = spark.read.options(sep=',', header=True, inferSchema=True).csv("/home/ali/DataScience/lax_passengers_header.csv").select('ReportPeriod', 'Passenger_Count')
port_df.show(5)

+--------------------+---------------+
|        ReportPeriod|Passenger_Count|
+--------------------+---------------+
|01/01/2006 12:00:...|            490|
|01/01/2006 12:00:...|            498|
|01/01/2006 12:00:...|            753|
|01/01/2006 12:00:...|            688|
|01/01/2006 12:00:...|         401535|
+--------------------+---------------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import *
split_column = split(port_df['ReportPeriod'], ' ').alias('date')
split_date = split(split_column[0], '/').alias('date')
port_df = port_df.select(split_date[2].alias('year'), split_date[0].alias('month'), 'Passenger_Count')
port_df.show()

+----+-----+---------------+
|year|month|Passenger_Count|
+----+-----+---------------+
|2006|   01|            490|
|2006|   01|            498|
|2006|   01|            753|
|2006|   01|            688|
|2006|   01|         401535|
|2006|   01|         389745|
|2006|   01|            561|
|2006|   01|          98991|
|2006|   01|         163067|
|2006|   01|          93672|
|2006|   01|         156751|
|2006|   01|         121649|
|2006|   01|          26585|
|2006|   01|         120111|
|2006|   01|          60948|
|2006|   01|         381419|
|2006|   01|          68348|
|2006|   01|         374238|
|2006|   01|          42256|
|2006|   01|         135622|
+----+-----+---------------+
only showing top 20 rows



In [5]:
group_port_df = port_df.groupBy('year', 'month').agg(sum('Passenger_Count').alias('Passenger_Count')).filter('Passenger_Count > 5000000').orderBy(col('year'), col('month'))
group_port_df.show()

+----+-----+---------------+
|year|month|Passenger_Count|
+----+-----+---------------+
|2006|   03|        5090294|
|2006|   04|        5087874|
|2006|   05|        5177408|
|2006|   06|        5510023|
|2006|   07|        5919723|
|2006|   08|        5745243|
|2006|   12|        5072332|
|2007|   03|        5241814|
|2007|   04|        5173114|
|2007|   05|        5256929|
|2007|   06|        5626463|
|2007|   07|        6052849|
|2007|   08|        5939548|
|2007|   10|        5070042|
|2007|   12|        5187718|
|2008|   03|        5233679|
|2008|   05|        5247987|
|2008|   06|        5539873|
|2008|   07|        5791568|
|2008|   08|        5715285|
+----+-----+---------------+
only showing top 20 rows



In [8]:
for row in group_port_df.collect():
    print(f'{row.year}/{row.month}   {row.Passenger_Count}')

2006/03   5090294
2006/04   5087874
2006/05   5177408
2006/06   5510023
2006/07   5919723
2006/08   5745243
2006/12   5072332
2007/03   5241814
2007/04   5173114
2007/05   5256929
2007/06   5626463
2007/07   6052849
2007/08   5939548
2007/10   5070042
2007/12   5187718
2008/03   5233679
2008/05   5247987
2008/06   5539873
2008/07   5791568
2008/08   5715285
2009/06   5040491
2009/07   5604410
2009/08   5437442
2010/06   5311394
2010/07   5692861
2010/08   5617846
2010/12   5049857
2011/04   5028216
2011/05   5477064
2011/06   5631716
2011/07   6071342
2011/08   5918524
2011/10   5124604
2011/12   5028644
2012/03   5252557
2012/04   5161128
2012/05   5412946
2012/06   5739904
2012/07   6190326
2012/08   6093345
2012/09   5054257
2012/10   5191471
2012/12   5278513
2013/01   5050736
2013/03   5545523
2013/04   5310497
2013/05   5713956
2013/06   6114514
2013/07   6468978
2013/08   6330168
2013/09   5308253
2013/10   5515805
2013/11   5057005
2013/12   5769931
2014/01   5419617
2014/03   